In [1]:
from model import Model
import numpy as np
import os
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import Dataset,DataLoader
from torchvision.transforms import ToTensor
import pandas as pd
from pandas import DataFrame
import torch.nn as nn
import cv2
import torch.optim as optim
from sklearn.utils import shuffle

In [2]:
class ResNet18(nn.Module):
    def __init__(self,**kwargs):
        super(ResNet18,self).__init__()
        resnet50 = models.resnet50(pretrained=True)
        self.base=resnet50
        self.fc1 = nn.Linear(1000,128)
        self.fc2 = nn.Linear(128,4)
        self.relu=nn.ReLU()


    def forward(self,x):
        x1 = self.base(x)
        out=self.fc1(x1)
        out=self.relu(out)
        out1=self.fc2(out)
        out1=self.relu(out1)
        #out = self.relu(out)
    
        return out1,out

model = ResNet18().cuda()
model.load_state_dict(torch.load("bone_resnet18.pth"))

<All keys matched successfully>

In [3]:
normalize = transforms.Normalize(mean=[0.45271412, 0.45271412, 0.45271412],
                                     std=[0.33165374, 0.33165374, 0.33165374])

transformer = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    normalize
])

In [4]:
linchuang_data=pd.read_excel("linchuang.xlsx")
for i in range(len(linchuang_data)):
    if linchuang_data.iloc[i,5]=='男':
        linchuang_data.iloc[i,5]=1
    else:
        linchuang_data.iloc[i,5]=0
info_dict=[]
for i in range(len(linchuang_data)):
    info_dict.append([linchuang_data.iloc[i,1],linchuang_data.iloc[i,2:].tolist()])
info_dict=DataFrame(info_dict)
info_dict.columns=["sfz","clinical"]
im_feat_list=[]
for s in range(len(info_dict)):
    sfz=info_dict.loc[s,"sfz"]
    xray_dir="./0308xray/"+sfz+"/术后/"
    if not os.path.exists(xray_dir):
        xray_dir=xray_dir.replace("后","前")
    # print(xray_dir)
    xray=os.listdir(xray_dir)[0]
    xray=os.path.join(xray_dir,xray)
    xray_img=Image.open(xray)
    xray_img=transformer(xray_img)
    xray_img=xray_img.unsqueeze(0)
    model.eval()
    _,img_feature=model(xray_img.cuda())
    im_feat_list.append(img_feature.cpu().detach().numpy()[0])
info_dict["img_feature"]=im_feat_list
info_dict

,sfz,clinical,img_feature
0,332627197802060957,"[44, 54, 166, 1, 90, 0, 0]","[0.0, 0.81039315, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8..."
1,341221198504228091,"[37, 71, 173, 1, 60, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6311585,..."
2,310223196309290022,"[58, 51, 152, 0, 180, 1, 0]","[0.18839669, 1.1271399, 0.0, 0.0, 0.0, 0.0, 0...."
3,432801194906183019,"[73, 49, 165, 1, 1, 1, 0]","[0.0, 0.6890219, 0.0, 0.0, 0.0, 0.0, 0.0, 1.62..."
4,310106195111290020,"[70, 62, 160, 0, 180, 2, 0]","[0.0, 0.21370673, 0.0, 0.0, 0.0, 0.0, 0.0, 1.7..."
5,410702198607212514,"[36, 71, 180, 1, 60, 20, 0]","[0.0, 0.0, 0.0, 0.0, 2.2237473, 4.076602, 0.0,..."
6,320684198611114690,"[35, 85, 170, 1, 1, 32, 3]","[0.0, 4.076841, 0.0, 0.014424676, 2.996078, 2...."
7,321083197012192416,"[51, 88, 170, 1, 90, 1, 0]","[0.0, 0.7704218, 0.0, 0.0, 0.0, 0.0, 0.0, 1.87..."
8,31022919640102362X,"[58, 51, 160, 0, 180, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4764464,..."
9,310102196203214868,"[60, 74, 172, 0, 1, 32, 3]","[0.0, 4.8046637, 0.0, 0.0, 2.9886243, 2.652146..."


In [5]:
drop_list=[]
for i in range(len(info_dict)):
    sfz=info_dict.iloc[i,0]
    plan_dir="../Multidimensional-time-series-with-transformer-main/0307data/"+sfz+"/tb_sub_plan.xlsx"
    if not os.path.exists(plan_dir):
        drop_list.append(i)
info_dict.drop(drop_list,axis=0,inplace=True)
info_dict.index=range(len(info_dict))
info_dict

,sfz,clinical,img_feature
0,310223196309290022,"[58, 51, 152, 0, 180, 1, 0]","[0.18839669, 1.1271399, 0.0, 0.0, 0.0, 0.0, 0...."
1,320684198611114690,"[35, 85, 170, 1, 1, 32, 3]","[0.0, 4.076841, 0.0, 0.014424676, 2.996078, 2...."
2,31022919640102362X,"[58, 51, 160, 0, 180, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4764464,..."
3,310102196203214868,"[60, 74, 172, 0, 1, 32, 3]","[0.0, 4.8046637, 0.0, 0.0, 2.9886243, 2.652146..."
4,412327196409130022,"[57, 63, 165, 0, 270, 11, 1]","[0.0, 0.0, 0.0, 0.67695, 0.86911255, 5.204607,..."
5,210603195412156022,"[67, 48, 155, 0, 360, 0, 0]","[0.18312973, 0.40110716, 0.0, 0.0, 0.37218925,..."
6,330382200511251711,"[15, 60, 187, 1, 1, 11, 0]","[0.0, 0.116981946, 0.0, 0.0, 0.0, 0.0, 0.0, 1...."
7,310104196506112810,"[57, 81, 180, 1, 8, 31, 3]","[0.0, 5.8020372, 0.0, 0.0, 3.2066607, 2.943308..."
8,330304199611237816,"[24, 70, 175, 1, 360, 11, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.188794, ..."
9,321088198904010856,"[33, 65, 170, 1, 180, 2, 0]","[0.0, 0.5156565, 0.0, 0.0, 0.15082756, 0.0, 0...."


In [6]:
plan_list=[]
for i in range(len(info_dict)):
    sfz=info_dict.iloc[i,0]
    plan_dir="../Multidimensional-time-series-with-transformer-main/0307data/"+sfz+"/tb_sub_plan.xlsx"
    plan=pd.read_excel(plan_dir)
    # print(plan.load[0])
    plan_list.append([len(plan),plan.load[0]])
info_dict["plan"]=plan_list
info_dict

,sfz,clinical,img_feature,plan
0,310223196309290022,"[58, 51, 152, 0, 180, 1, 0]","[0.18839669, 1.1271399, 0.0, 0.0, 0.0, 0.0, 0....","[6, 24]"
1,320684198611114690,"[35, 85, 170, 1, 1, 32, 3]","[0.0, 4.076841, 0.0, 0.014424676, 2.996078, 2....","[14, 19]"
2,31022919640102362X,"[58, 51, 160, 0, 180, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4764464,...","[6, 24]"
3,310102196203214868,"[60, 74, 172, 0, 1, 32, 3]","[0.0, 4.8046637, 0.0, 0.0, 2.9886243, 2.652146...","[10, 9]"
4,412327196409130022,"[57, 63, 165, 0, 270, 11, 1]","[0.0, 0.0, 0.0, 0.67695, 0.86911255, 5.204607,...","[17, 20]"
5,210603195412156022,"[67, 48, 155, 0, 360, 0, 0]","[0.18312973, 0.40110716, 0.0, 0.0, 0.37218925,...","[6, 22]"
6,330382200511251711,"[15, 60, 187, 1, 1, 11, 0]","[0.0, 0.116981946, 0.0, 0.0, 0.0, 0.0, 0.0, 1....","[27, 4]"
7,310104196506112810,"[57, 81, 180, 1, 8, 31, 3]","[0.0, 5.8020372, 0.0, 0.0, 3.2066607, 2.943308...","[12, 66]"
8,330304199611237816,"[24, 70, 175, 1, 360, 11, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.188794, ...","[6, 18]"
9,321088198904010856,"[33, 65, 170, 1, 180, 2, 0]","[0.0, 0.5156565, 0.0, 0.0, 0.15082756, 0.0, 0....","[6, 12]"


In [7]:
class Clinical(Dataset):
    def __init__(self,df,transform=None):
        self.num_cls = len(df)
        self.img_list = []
        # for i in range(self.num_cls):
        #     print(df.iloc[i,1])\
        #     self.img_list += [[df.iloc[i,1].tolist().extend(df.iloc[i,2]),df.iloc[i,3]]]
        self.img_list=df
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

#         img_path = self.img_list[idx][1]
        # image = self.img_list[idx][0].convert('RGB')

        # if self.transform:
        #     image = self.transform(image)
        sample = {'sfz': self.img_list[idx][0],
                  'clinical': self.img_list[idx][1],
                  'img_ft': self.img_list[idx][2],
                  'plan': int(self.img_list[idx][3])}
        return sample

In [8]:
# clinical_data=[]
# for i in range(len(info)):
#     sfz=info.iloc[i,0]
#     plan_dir="../Multidimensional-time-series-with-transformer-main/0307data/"+sfz+"/tb_sub_plan.xlsx"
#     if os.path.exists(plan_dir):
#         plan=pd.read_excel(plan_dir)
#         # print(plan.load[0])
#         clinical_data.append([info.iloc[i,0],info.iloc[i,1],info.iloc[i,2],[len(plan),plan.load[0]]])

# clinical_data=DataFrame(clinical_data)
# clinical_data

In [9]:
# info=pd.read_excel("patient_info.xlsx")
# img_feature=np.load("xray_feature.npy")
# info["xray"]=img_feature.tolist()
# info


In [32]:
train_df=info_dict.iloc[:12,:]
test_df=info_dict.iloc[12:,:]
test_df.index=range(len(test_df))

device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 4
# train_dataset = mnist.MNIST(root='./train', train=True, transform=ToTensor())
# test_dataset = mnist.MNIST(root='./test', train=False, transform=ToTensor())
train_dataset=Clinical(train_df)
test_dataset=Clinical(test_df)
# print(test_dataset[0][0].shape)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [33]:
def train(optimizer, epoch,bs):
    model.train()
    train_loss = 0
    for batch_index, batch_samples in enumerate(train_loader):
        clinical_f,img_f,wb_plan= batch_samples['clinical'].to(device),batch_samples['img_ft'].to(device), batch_samples['plan'].to(device)
        optimizer.zero_grad()
        output = model(clinical_f,img_f)
        criteria = nn.MAELoss()
        loss = criteria(output,wb_plan)
        train_loss += loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if batch_index % (3) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]    Train Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
    
    # if epoch%10==0: torch.save(model.state_dict(),'bone_resnet18.pth')
    print('Train set: Average loss: {:.4f}'.format(
        train_loss/len(train_loader.dataset)))
    # if train_correct / len(train_loader.dataset)>=0.99: torch.save(model.state_dict(),'bone_resnet18.pth')
    return train_loss/len(train_loader.dataset)

In [35]:
batchsize=4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Model().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
total_epoch = 10
train_num=range(1,total_epoch+1)
avg_loss=[]
for epoch in range(1, total_epoch+1):
    loss=train(optimizer,epoch,batchsize)
    avg_loss.append(loss)



KeyError: 0

In [127]:
# for current_epoch in range(all_epoch):
#     model.train()
#     for idx, (train_x, train_label) in enumerate(train_loader):
#         train_x = train_x.to(device)
#         # print(train_x.size())
#         train_label = train_label.to(device)
#         sgd.zero_grad()
#         predict_y = model(train_x.float())
#         loss = loss_fn(predict_y, train_label.long())
#         loss.backward()
#         sgd.step()

#     all_correct_num = 0
#     all_sample_num = 0
#     model.eval()
    
#     for idx, (test_x, test_label) in enumerate(test_loader):
#         test_x = test_x.to(device)
#         test_label = test_label.to(device)
#         predict_y = model(test_x.float()).detach()
#         predict_y =torch.argmax(predict_y, dim=-1)
#         current_correct_num = predict_y == test_label
#         all_correct_num += np.sum(current_correct_num.to('cpu').numpy(), axis=-1)
#         all_sample_num += current_correct_num.shape[0]
#     acc = all_correct_num / all_sample_num
#     print('accuracy: {:.3f}'.format(acc))
#     if not os.path.isdir("models"):
#         os.mkdir("models")
#     torch.save(model, 'models/mnist_{:.3f}.pkl'.format(acc))
#     # if np.abs(acc - prev_acc) < 1e-4:
#     #     break
#     prev_acc = acc
# print("Model finished training")

In [138]:
def train(optimizer, epoch):
    
    model.train()
    
    train_loss = 0
    train_correct = 0
    
    for batch_index, batch_samples in enumerate(train_loader):
        
        data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)

        optimizer.zero_grad()
        output = model(data)
        # pred = output.argmax(dim=1, keepdim=True)
        criteria = nn.CrossEntropyLoss()
        # print("output:",output,output.shape)
        # print(pred)
        loss = criteria(output, target.long())

        train_loss += criteria(output, target.long())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        # print("pred:",pred)
        # train_correct += pred.eq(target.long().view_as(pred)).sum().item()
    

        if batch_index % (3) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
    
    # if epoch%10==0: torch.save(model.state_dict(),'bone_resnet18.pth')
    print('\nTrain set: Average loss: {:.4f}\n'.format(
        train_loss/len(train_loader.dataset)))
    if train_correct / len(train_loader.dataset)>=0.99: torch.save(model.state_dict(),'bone_resnet18.pth')
    return train_loss/len(train_loader.dataset)

In [139]:
def val(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
       
    criteria = nn.CrossEntropyLoss()
    with torch.no_grad():

        predlist=[]
        # scorelist=[]
        targetlist=[]
        for batch_index, batch_samples in enumerate(val_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
            
            output = model(data)
            
            test_loss += criteria(output, target.long())
            # score = F.softmax(output, dim=1)
            # pred = output.argmax(dim=1, keepdim=True)

            # correct += pred.eq(target.long().view_as(pred)).sum().item()

            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, output.cpu().numpy())
            # scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
           
    return targetlist,predlist

In [140]:
def test(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
    results = []
    
    criteria = nn.CrossEntropyLoss()
    with torch.no_grad():
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        feature_list=[]
        for batch_index, batch_samples in enumerate(test_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
            output = model(data)
            # print(feature.cpu().numpy().shape)
            test_loss += criteria(output, target.long())

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.long().view_as(pred)).sum().item()
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
            feature_list.extend(feature.cpu().tolist())

    print(type(feature_list))
    return targetlist, predlist

In [4]:
cccdf = pd.read_excel("../lstm-gru-pytorch/clinical_with_plan2.xlsx")
cccdf = shuffle(cccdf)
for i in range(len(cccdf)):
    cccdf.iloc[i,0] = str(cccdf.iloc[i,0])
cccdf.to_excel("./clinical_with_plan_shuffle.xlsx",index = False)